# Masked-Language Modeling With BERT
Đầu vào là một câu chưa hoàn chỉnh (bị che đi một số phần) và yêu cầu BERT hoàn thành câu.

Quá trình MLM (Masked-Language Modeling) hoạt động:
1. Tokenize text, nhận được tensors:
    * *input_ids*
    * *token_type_ids*
    * *attention_mask*

    Trong MLM, ta không sử dụng *token_type_ids*

2. Tạo *labels* tensor để tính **loss** và tối ưu hóa bằng cách sao chép *input_ids*
3. Che (mask) 15% tokens trong *input_ids*
4. Tính toán **loss** dựa vào *labels* và *input_ids*


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW
from transformers import TrainingArguments
from transformers import Trainer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


### Tokenize text
Thu được:
* *input_ids*
* *token_type_ids*
* *attention_mask*

In [4]:
with open('./text_v1.txt', 'r') as fp:
    text = fp.read().split('\n')
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')



tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]])

### Tạo labels
Copy từ *input_ids*

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.input_ids

### Masking
Che ngẫu nhiên token trong *input_ids* với xác suất 15%.

Do không muốn đặt MASK vào các token như *CLS*, *SEP* và *PAD* (101,102,0), ta bổ sung các điều kiện cho *mask_arr*

In [5]:

# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)



Sử dụng *mask_arr* để chỉ nơi đặt MASK tokens (Có giá trị *True*) với token id là 103

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103


### Khởi tạo dữ liệu

In [6]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = MeditationsDataset(inputs)
loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

### Train mô hình

In [9]:
from tqdm import tqdm  # for our progress bar
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()
epochs = 2
optim = AdamW(model.parameters(), lr=5e-5)
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/254 [00:00<?, ?it/s]/tmp/ipykernel_1988224/2089794389.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|██████████| 254/254 [00:51<00:00,  4.92it/s, loss=0.447]  


In [9]:
args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=2,
    num_train_epochs=2
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

trainer.train()

/home/phung/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 507
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 254
  0%|          | 0/254 [00:00<?, ?it/s]/tmp/ipykernel_1992366/2089794389.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/home/phung/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: U

{'train_runtime': 234.6359, 'train_samples_per_second': 4.322, 'train_steps_per_second': 1.083, 'train_loss': 0.5388064046544353, 'epoch': 2.0}


TrainOutput(global_step=254, training_loss=0.5388064046544353, metrics={'train_runtime': 234.6359, 'train_samples_per_second': 4.322, 'train_steps_per_second': 1.083, 'train_loss': 0.5388064046544353, 'epoch': 2.0})